<a href="https://colab.research.google.com/github/afullhart/climateanalogs/blob/main/Colab/Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%reset -f

In [ ]:
!pip install rioxarray
!pip install rasterio
!pip install tifffile
!pip install imagecodecs

In [37]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from sklearn.metrics import silhouette_samples, silhouette_score
import rasterio
import rioxarray as rx
from PIL import Image
import tifffile
import imagecodecs
import os

comf = '/content/drive/My Drive/Colab Notebooks/Analogs/Extract_Composite.tif'
silhf = '/content/drive/My Drive/Colab Notebooks/Analogs/silhouette'

image = Image.open( '/content/drive/My Drive/Colab Notebooks/Analogs/IsoCluster.tif')
arr = np.asarray(image)
mask = arr > 0

img_array = tifffile.imread(comf)
shp = img_array.shape
new_arr = []
for i in range(shp[2]):
  image = Image.fromarray(img_array[:, :, i]).convert('L')
  arr = np.asarray(image)
  filtered_arr = arr[mask].flatten().tolist()
  new_arr.append(filtered_arr)

new_arr = np.array(new_arr).transpose()

X = new_arr

test_cluster_sizes = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]
for n in test_cluster_sizes:

  image = Image.open(os.path.join(silhf, 'IsoCluster_{}.tif'.format(n)))
  arr = np.asarray(image)
  filtered_arr = arr[mask]
  filtered_labels = filtered_arr.flatten()

  cluster_labels = filtered_labels
  print(cluster_labels.shape)

  silhouette_avg = silhouette_score(X, cluster_labels, sample_size=50000)
  print(silhouette_avg)



(1622533,)
-0.42349999811106914
(1622533,)
-0.07715996891191709
(1622533,)
-0.19894
(1622533,)
-0.13867998863636363
(1622533,)
-0.049859839807769334
(1622533,)
-0.09009996656726148
(1622533,)
-0.07493994910941476
(1622533,)
-0.03768
(1622533,)
-0.09088
(1622533,)
-0.02168


In [16]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from sklearn.metrics import silhouette_samples, silhouette_score
import rasterio
import rioxarray as rx
from PIL import Image
import tifffile
import imagecodecs

comf = '/content/drive/My Drive/Colab Notebooks/Analogs/Extract_Composite.tif'
isof = '/content/drive/My Drive/Colab Notebooks/Analogs/IsoCluster.tif'

image = Image.open(isof)
arr = np.asarray(image)
mask = arr > 0
filtered_arr = arr[mask]
filtered_labels = filtered_arr.flatten()

img_array = tifffile.imread(comf)
shp = img_array.shape
new_arr = []
for i in range(shp[2]):
  image = Image.fromarray(img_array[:, :, i]).convert('L')
  arr = np.asarray(image)
  filtered_arr = arr[mask].flatten().tolist()
  new_arr.append(filtered_arr)

new_arr = np.array(new_arr).transpose()

X = new_arr
cluster_labels = filtered_labels

print(X.shape)
print(cluster_labels.shape)

silhouette_avg = silhouette_score(X, cluster_labels, sample_size=50000)
print(silhouette_avg)


(1622533, 23)
(1622533,)
-0.10031994805194806


In [ ]:
# Authors: The scikit-learn developers
# SPDX-License-Identifier: BSD-3-Clause

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(
    n_samples=500,
    n_features=2,
    centers=4,
    cluster_std=1,
    center_box=(-10.0, 10.0),
    shuffle=True,
    random_state=1,
)  # For reproducibility

range_n_clusters = [2, 3, 4, 5, 6]


print(X.size)
print(X.shape)

for n_clusters in range_n_clusters:

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    #print(cluster_labels)
    print(cluster_labels.size)
    print(cluster_labels.shape)

1000
(500, 2)
For n_clusters = 2 The average silhouette_score is : 0.7049787496083262
500
(500,)
For n_clusters = 3 The average silhouette_score is : 0.5882004012129721
500
(500,)
For n_clusters = 4 The average silhouette_score is : 0.6505186632729437
500
(500,)
For n_clusters = 5 The average silhouette_score is : 0.561464362648773
500
(500,)
For n_clusters = 6 The average silhouette_score is : 0.4857596147013469
500
(500,)
